In [1]:
import tensorflow as tf
from utils.modelLoader import ModelLoader
import pandas as pd
from utils.utilities import buildRunName


In [22]:
metaData = pd.read_json("../data_meta/meta_all_str.json")
temp = pd.DataFrame(pd.get_dummies(metaData["Age"]))
temp["combined"] = temp[["0","1",2,3,4,5,6,7,8,9,10]].values.tolist()
metaData["Age"] = temp["combined"]
# metaData["Age"] = metaData["Age"].astype(str)
# metaData["Mask"] = metaData["Mask"].astype(str)
# metaData["Face"] = metaData["Face"].astype(str)
# filenames = tf.constant(metaData.iloc[:, 0].tolist())
# labels = tf.constant(metaData.iloc[:, 1:].values)

# dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))


# def _parse_function(filename, labels):
#     image = tf.io.read_file(filename)
#     image = tf.image.decode_jpeg(
#         image, channels=3
#     )  # Channels needed because some test images are b/w
#     image = tf.image.resize(image, (224, 224))
#     image.set_shape((224, 224, 3))
#     return image, labels


# dataset = dataset.map(_parse_function)

train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0 / 255, validation_split=0.2
)

train_images = train_generator.flow_from_dataframe(
    dataframe=metaData,
    x_col="Filepath",
    y_col=["Face", "Mask", "Age"],
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="raw",
    batch_size=32,
    shuffle=True,
    seed=123,
    subset="training",
)


val_images = train_generator.flow_from_dataframe(
    dataframe=metaData,
    x_col="Filepath",
    y_col=["Face", "Mask", "Age"],
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="raw",
    batch_size=32,
    shuffle=True,
    seed=123,
    subset="validation",
)

model = ModelLoader().loadMobileNetV1Multi(10)

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss={
        "out_age_prediction": tf.keras.losses.SparseCategoricalCrossentropy(
            ignore_class=0
        ),
        "out_face_detection": tf.keras.losses.CategoricalCrossentropy(),
        "out_mask_detection": tf.keras.losses.CategoricalCrossentropy(),
    },
    loss_weights=[1 / 3, 1 / 3, 1 / 3],
    metrics={
        "out_age_prediction": tf.keras.metrics.Accuracy(),
        "out_face_detection": tf.keras.metrics.Accuracy(),
        "out_mask_detection": tf.keras.metrics.Accuracy(),
    },
)

log_dir = "../logs/fit/" + buildRunName("MobileNet_Multi_Dropout-20", 10, 32)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = model.fit(
    train_images,
    # y = {"out_face_detection": metaData["Face"].to_numpy(),
    #      "out_mask_detection": metaData["Mask"].to_numpy(),
    #      "out_age_prediction": metaData["Age"].to_numpy()
    # },
    validation_data=val_images,
    epochs=10,
    callbacks=[tensorboard_callback],
)

model.save("../models/" + buildRunName("MobileNet_Multi_Dropout-20", 10, 32))


KeyError: '[1, 2, 3, 4, 5] not in index'